# Semantic Kernel  & Agent-to-Agent (A2A) Protocol

This notebook demonstrates how to implement Agent-to-Agent (A2A) communication using Semantic Kernel. We'll create two agents:

1. **Flight Booking Agent** - A specialized agent that handles flight bookings
2. **Travel Planning Agent** - A general travel agent that uses the flight booking agent as a tool

## What does Agent-to-Agent (A2A) Protocol offer?

A2A Protocol allows different AI agents to work together by calling each other's services. This creates a distributed system where:

- Each agent can specialize in specific tasks
- Agents can leverage other agents' capabilities
- The system becomes more modular and scalable

## Architecture Overview

```
User Request → Travel Planning Agent → Flight Booking Agent → Response
```

The Travel Planning Agent acts as an orchestrator that can handle various travel-related tasks, and when it needs to book flights, it communicates with the specialized Flight Booking Agent through the A2A protocol.


## Prerequisites and Setup

First, let's install the required dependencies and set up our environment.


In [ ]:
# Install required packages
# If you already did the uv configuration, you can skip this step
#%pip install semantic-kernel python-dotenv fastapi uvicorn httpx a2a-sdk

In [ ]:
# Import necessary libraries
import os
import logging
import threading
from uuid import uuid4

# Third-party imports
import httpx
import uvicorn
from dotenv import load_dotenv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv('../.env')

print("✅ Dependencies installed and environment configured!")


## Flight Booking Agent Implementation

The Flight Booking Agent is a specialized agent that handles flight booking requests. It understands user requests and maintain conversation context.


### Understanding the Flight Booking Agent Architecture

Before we dive into the implementation, let's understand what makes this agent special:

1. **Semantic Kernel Integration**: We use Semantic Kernel's `ChatCompletionAgent` to instantiate our Agent
2. **Context Management**: Each conversation is tracked separately using context IDs, allowing multiple users or sessions
4. **Conversation History**: The agent maintains memory of previous interactions within each context

The agent follows a stateful conversation pattern where it:
- Gathers flight booking information progressively
- Maintains context across multiple turns
- Provides confirmations and summaries



In [ ]:
# Import Semantic Kernel components
from semantic_kernel.agents.chat_completion.chat_completion_agent import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.chat_history import ChatHistory

class SemanticKernelFlightBookingAgent:
    """A flight booking agent using Semantic Kernel and Azure OpenAI."""

    def __init__(self):
        """Initialize the flight booking agent with Azure OpenAI service."""
        logger.info("Initializing SemanticKernelFlightBookingAgent.")

        self.chat_agent = ChatCompletionAgent(
            service=AzureChatCompletion(),
            name="FlightBookingAssistant",
            instructions=(
                "You are a helpful flight booking assistant. "
                "Your task is to help users book flights by gathering necessary information "
                "such as departure city, destination city, travel dates, number of passengers, "
                "and preferred class of service. Once you have all the required information, "
                "provide a confirmation summary and simulate a successful booking."
            )
        )

        # Store chat history per context to maintain conversation state
        self.history_store: dict[str, ChatHistory] = {}

        logger.info("SemanticKernelFlightBookingAgent initialized successfully.")

    def _get_or_create_chat_history(self, context_id: str) -> ChatHistory:
        """Get existing chat history or create a new one for the given context."""
        chat_history = self.history_store.get(context_id)

        if chat_history is None:
            chat_history = ChatHistory(
                messages=[],
                system_message=(
                    "You are a helpful flight booking assistant. "
                    "Help users book flights by gathering all necessary information: "
                    "departure city, destination city, travel dates, number of passengers, "
                    "and preferred class. Once you have complete information, "
                    "provide a booking confirmation summary."
                )
            )
            self.history_store[context_id] = chat_history
            logger.info(f"Created new ChatHistory for context ID: {context_id}")

        return chat_history

    async def chat(self, user_input: str, context_id: str) -> str:
        """
        Process a flight booking request from the user.

        Args:
            user_input: The user's request for flight booking
            context_id: The context ID for maintaining conversation state

        Returns:
            The response from the flight booking agent
        """
        logger.info(f"Received flight booking request: {user_input} with context ID: {context_id}")

        if not user_input or not user_input.strip():
            logger.error("User input is empty.")
            raise ValueError("User input cannot be empty.")

        try:
            # Get or create chat history for the context
            chat_history = self._get_or_create_chat_history(context_id)

            # Add user input to chat history
            chat_history.messages.append(
                ChatMessageContent(role="user", content=user_input))

            # Create a new thread from the chat history
            thread = ChatHistoryAgentThread(
                chat_history=chat_history, thread_id=str(uuid4()))

            # Get response from the agent
            response = await self.chat_agent.get_response(message=user_input, thread=thread)

            # Add assistant response to chat history
            chat_history.messages.append(ChatMessageContent(
                role="assistant", content=response.content.content))


            logger.info(f"Chat history length is {len(chat_history.messages)} messages for context ID: {context_id}")
            logger.info(f"Flight booking agent response: {response.content.content}")

            return response.content.content

        except Exception as e:
            logger.error(f"Error processing flight booking request: {e}")
            return f"I apologize, but I encountered an error while processing your flight booking request: {str(e)}"

# Create an instance of the flight booking agent
flight_booking_agent = SemanticKernelFlightBookingAgent()
print("✅ Flight Booking Agent created successfully!")


### Test the Flight Booking Agent

Let's test our flight booking agent directly to see how it works:


In [ ]:
response = await flight_booking_agent.chat(  
    user_input="I want to book a flight from New York to London on July 15th", 
    context_id="test_context_1"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

### What Just Happened?

In the test above, notice how the agent:

1. **Extracted Information**: It identified the departure city (New York), destination (London), and date (July 15th) from the user's natural language input
2. **Identified Missing Information**: It recognized that it still needed the number of passengers and class preference
3. **Maintained Context**: It structured the response in a clear, organized way
4. **Guided the Conversation**: It asked for the remaining information in a user-friendly manner

This demonstrates the agent's ability to understand partial information and guide users through a complete booking process.


### Completing the Booking Flow

Let's continue the conversation by providing the missing information:


In [ ]:
response = await flight_booking_agent.chat(
    user_input="I need it for 2 passengers, business class, mock the rest",
    context_id="test_context_1"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

### Conversation Continuity in Action

Perfect! The agent:

1. **Remembered the Context**: It recalled all the previous information from our conversation
2. **Processed New Information**: It understood "2 passengers" and "business class"
3. **Provided Confirmation**: It summarized all the booking details
4. **Simulated Success**: It confirmed the booking as if it were a real transaction

This shows how the conversation history is maintained within the same context ID (`test_context_1`).


---
### Testing Context Isolation

Now let's test with a different context ID to see how conversation contexts are isolated:


In [ ]:
response = await flight_booking_agent.chat(
        "Book a flight to Paris tomorrow", 
        "test_context_2"
    )
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

### Context Isolation Explained

Notice how using `test_context_2` created a completely fresh conversation:

- **No Memory of Previous Booking**: The agent doesn't remember the New York to London booking
- **Fresh Start**: It treats this as a new conversation and asks for all required information
- **Independent State**: Each context maintains its own conversation history

This is crucial for multi-user scenarios where different users or sessions need isolated conversations.


---

### Understanding A2A Protocol Components

Now that we have a working flight booking agent, we need to make it available to other agents through the A2A protocol. This requires several components:

#### 1. **Agent Executor**
- Bridges between the A2A protocol and our agent
- Handles the conversion of A2A requests to agent method calls
- Manages the event queue for responses

#### 2. **Agent Card**
- Describes the agent's capabilities in a standardized format
- Lists available skills and their descriptions
- Provides examples of how to use the agent

#### 3. **A2A Server**
- Exposes the agent via HTTP endpoints
- Handles A2A protocol communication
- Manages requests and responses

Let's implement these components step by step, starting with the **Agent Executor**



In [ ]:
# Import A2A SDK components
from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events import EventQueue
from a2a.utils import new_agent_text_message, new_task

class SemanticKernelFlightBookingAgentExecutor(AgentExecutor):
    """Executor for SemanticKernelFlightBookingAgent that handles A2A protocol integration."""

    def __init__(self):
        """Initialize the executor with a flight booking agent instance."""
        logger.info("Initializing SemanticKernelFlightBookingAgentExecutor.")
        self.agent = SemanticKernelFlightBookingAgent()
        logger.info("SemanticKernelFlightBookingAgentExecutor initialized successfully.")

    async def execute(
        self,
        context: RequestContext,
        event_queue: EventQueue,
    ) -> None:
        """
        Execute a flight booking request.

        Args:
            context: The request context containing user input and task information
            event_queue: Queue for sending events and responses
        """
        user_input = context.get_user_input()
        task = context.current_task
        context_id = context.context_id

        # Create a new task if one doesn't exist
        if not task:
            task = new_task(context.message)
            await event_queue.enqueue_event(task)

        logger.info(f"Executing flight booking - User input: {user_input}, Task ID: {task.id}, Context ID: {context_id}")

        try:
            # Process the flight booking request
            result = await self.agent.chat(user_input, context_id)

            # Send the result back through the event queue
            await event_queue.enqueue_event(new_agent_text_message(result))

            logger.info("Flight booking executed successfully.")

        except ValueError as ve:
            logger.error(f"Validation error during flight booking: {ve}")
            await event_queue.enqueue_event(
                new_agent_text_message(f"I need more information to help you book a flight: {str(ve)}")
            )

        except Exception as e:
            logger.error(f"Unexpected error during flight booking execution: {e}")
            await event_queue.enqueue_event(
                new_agent_text_message(
                    "I apologize, but I encountered an error while processing your flight booking request. "
                    "Please try again or contact support if the issue persists."
                )
            )

    async def cancel(
        self,
        context: RequestContext,
        event_queue: EventQueue
    ) -> None:
        """
        Handle cancellation requests.

        Args:
            context: The request context
            event_queue: Queue for sending events
        """
        logger.warning("Cancel operation requested but not supported for flight booking agent.")
        raise Exception('Cancel operation not supported for flight booking operations.')

print("✅ Agent Executor created successfully!")


### Agent Executor Explained

The `SemanticKernelFlightBookingAgentExecutor` class serves as a crucial bridge:

1. **Protocol Translation**: It converts A2A protocol requests into calls to our agent's `chat` method
2. **Context Management**: It extracts context information from A2A requests
3. **Event Handling**: It manages the event queue to send responses back to the calling agent
4. **Error Handling**: It provides graceful error handling for various failure scenarios

The key methods are:
- `execute()`: Processes incoming A2A requests and calls our agent
- `cancel()`: Handles cancellation requests (not implemented for this example)


---


### Agent Card and Server Configuration

The configuration functions we just created serve important purposes:

#### **Agent Skill Definition**
- Defines what the agent can do (`flight_booking`)
- Provides human-readable descriptions
- Includes example usage patterns
- Uses tags for categorization

#### **Agent Card**
- Acts like a "business card" for the agent
- Specifies capabilities (like streaming support)
- Defines input/output modes (text in our case)
- Provides discovery information for other agents

#### **Server Application**
- Combines the executor, task store, and agent card
- Creates HTTP endpoints for A2A communication
- Handles the low-level protocol details



In [ ]:
# Import A2A server components
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
)

# Server configuration
SERVER_HOST = "0.0.0.0"
SERVER_PORT = 9999

def create_flight_booking_skill() -> AgentSkill:
    """Create and return the flight booking skill configuration."""
    return AgentSkill(
        id='flight_booking',
        name='Flight Booking',
        description='Assists users in booking flights based on their requests.',
        tags=['flight', 'booking', 'travel'],
        examples=[
            'Book a flight from New York to London next Monday.',
            'I need a flight to Paris tomorrow morning.',
        ],
    )

def create_agent_card() -> AgentCard:
    """Create and return the agent card configuration."""
    return AgentCard(
        name='Semantic Kernel Flight Booking Agent',
        description='An agent that helps users book flights using semantic kernel capabilities.',
        capabilities=AgentCapabilities(streaming=True),
        url=os.environ.get('A2A_SERVER_URL'),
        version='1.0.0',
        defaultInputModes=['text'],
        defaultOutputModes=['text'],
        skills=[create_flight_booking_skill()],
        supportsAuthenticatedExtendedCard=False,
    )

def create_flight_booking_server() -> A2AStarletteApplication:
    """Create and configure the A2A server application."""
    # Initialize request handler with the flight booking agent executor
    request_handler = DefaultRequestHandler(
        agent_executor=SemanticKernelFlightBookingAgentExecutor(),
        task_store=InMemoryTaskStore(),
    )

    # Create and return the server application
    return A2AStarletteApplication(
        agent_card=create_agent_card(),
        http_handler=request_handler,
    )

print("✅ A2A Server configuration created successfully!")


### Starting the A2A Server

We're starting the server in a background thread so it doesn't block our notebook execution. The server will:

1. **Listen on port 9999** for incoming A2A requests
2. **Serve the agent card** at `/.well-known/agent.json` for discovery
3. **Handle A2A protocol messages** and route them to our agent
4. **Run as a daemon thread** so it stops when the notebook kernel stops

> **Note**: If you see an "address already in use" error, it means the server is already running from a previous execution.


In [ ]:
# Function to start the A2A server in a separate thread
def start_a2a_server():
    """Start the A2A server in a separate thread."""
    server = create_flight_booking_server()
    uvicorn.run(server.build(), host=SERVER_HOST, port=SERVER_PORT)

# Start the server in a background thread
server_thread = threading.Thread(target=start_a2a_server, daemon=True)
server_thread.start()


In [ ]:
async with httpx.AsyncClient() as client:
    response = await client.get(f"http://localhost:{SERVER_PORT}/.well-known/agent.json")
    if response.status_code == 200:
        print("✅ A2A Server is running successfully!")
        agent_card = response.json()
        print(f"Agent Name: {agent_card.get('name')}")
        print(f"Agent Description: {agent_card.get('description')}")
        print(f"Agent Skills: {[skill.get('name') for skill in agent_card.get('skills', [])]}")

### Server Status Verification

Great! The server is running and responding to requests. The agent card endpoint (`/.well-known/agent.json`) is a standard A2A discovery mechanism that allows other agents to:

1. **Discover capabilities** - What skills the agent offers
2. **Understand interfaces** - How to communicate with the agent
3. **Get examples** - Sample requests to help with usage
4. **Check compatibility** - Protocol version and supported features

Now our flight booking agent is ready to be used by other agents through the A2A protocol!

---

### Travel Planning Agent Architecture

Now we create the orchestrating agent that will use our flight booking agent as a tool:

#### **Agent Configuration:**
- **General Purpose**: Unlike the specialized flight booking agent, this agent handles various travel tasks
- **Tool Integration**: Has access to the `FlightBookingTool` for flight-related requests
- **Smart Routing**: Decides when to use the flight booking tool vs. handling requests directly
- **Context Awareness**: Maintains its own conversation history separate from the flight booking agent

#### **How It Decides When to Use the Flight Booking Tool:**
The travel agent uses Semantic Kernel's function calling capabilities. When it detects flight-related requests in the conversation, it automatically calls the `book_flight` function, which triggers A2A communication with our flight booking agent.


In [ ]:
# Import A2A client components
from a2a.client import A2ACardResolver, A2AClient
from a2a.types import MessageSendParams, SendMessageRequest
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Configuration
FLIGHT_BOOKING_AGENT_URL = os.getenv("A2A_SERVER_URL", "http://localhost:9999")

class FlightBookingTool:
    """Tool for booking flights using the flight booking agent."""

    @kernel_function(
        description="Book a flight using the flight booking agent",
        name="book_flight"
    )
    async def book_flight(self, user_input: str) -> str:
        """
        Book a flight using the external flight booking agent.

        Args:
            user_input: The user's flight booking request

        Returns:
            The response from the flight booking agent
        """
        try:
            async with httpx.AsyncClient() as httpx_client:
                # Resolve the agent card from the flight booking agent
                resolver = A2ACardResolver(
                    httpx_client=httpx_client, 
                    base_url=FLIGHT_BOOKING_AGENT_URL
                )
                agent_card = await resolver.get_agent_card()

                # Create A2A client
                client = A2AClient(
                    httpx_client=httpx_client,
                    agent_card=agent_card
                )

                # Prepare the request
                request = SendMessageRequest(
                    id=str(uuid4()),
                    params=MessageSendParams(
                        message={
                            "messageId": uuid4().hex,
                            "role": "user",
                            "parts": [{"text": user_input}],
                            "contextId": "travel_booking_context",
                        }
                    )
                )

                # Send the message to the flight booking agent
                response = await client.send_message(request)
                result = response.model_dump(mode='json', exclude_none=True)

                logger.info(f"Flight booking tool response: {result}")
                return result["result"]["parts"][0]["text"]

        except Exception as e:
            logger.error(f"Error booking flight: {e}")
            return f"Sorry, I encountered an error while trying to book your flight: {str(e)}"

print("✅ Flight Booking Tool created successfully!")


In [ ]:
# Travel Planning Agent Implementation
def create_travel_agent() -> ChatCompletionAgent:
    """Create and configure the travel planning agent."""
    return ChatCompletionAgent(
        service=AzureChatCompletion(),
        name="TravelPlanner",
        instructions=(
            "You are a helpful travel planning assistant. "
            "Use the provided tools to assist users with their travel plans. "
            "When users ask about flights, use the book_flight tool to help them. "
            "You can also provide general travel advice, recommendations, and assistance "
            "with other travel-related tasks."
        ),
        plugins=[FlightBookingTool()]
    )

# Global chat history store for the travel agent
travel_chat_history_store: dict[str, ChatHistory] = {}

def get_or_create_travel_chat_history(context_id: str) -> ChatHistory:
    """Get existing chat history or create a new one for the given context."""
    chat_history = travel_chat_history_store.get(context_id)

    if chat_history is None:
        chat_history = ChatHistory(
            messages=[],
            system_message=(
                "You are a travel planning assistant. "
                "Your task is to help the user with their travel plans, including booking flights."
            )
        )
        travel_chat_history_store[context_id] = chat_history
        logger.info(f"Created new ChatHistory for context ID: {context_id}")

    return chat_history

# Initialize the travel agent
travel_planning_agent = create_travel_agent()

async def chat_with_travel_agent(user_input: str, context_id: str = "default") -> str:
    """
    Handle chat requests with the travel planning agent.

    Args:
        user_input: The user's message
        context_id: Context identifier for maintaining chat history

    Returns:
        The agent's reply
    """
    logger.info(f"Received travel chat request: {user_input} with context ID: {context_id}")

    try:
        # Get or create chat history for the context
        chat_history = get_or_create_travel_chat_history(context_id)

        # Add user input to chat history
        chat_history.messages.append(
            ChatMessageContent(role="user", content=user_input))

        # Create a new thread from the chat history
        thread = ChatHistoryAgentThread(
            chat_history=chat_history, thread_id=str(uuid4()))

        # Get response from the agent
        response = await travel_planning_agent.get_response(message=user_input, thread=thread)

        # Add assistant response to chat history
        chat_history.messages.append(ChatMessageContent(
            role="assistant", content=response.content.content))

        logger.info(f"Travel agent response: {response.content.content}")

        return response.content.content

    except Exception as e:
        logger.error(f"Error processing travel chat request: {e}")
        return "Sorry, I encountered an error processing your request. Please try again."

print("✅ Travel Planning Agent created successfully!")


### Test the A2A Communication

Now let's test the Agent-to-Agent communication! We'll chat with the Travel Planning Agent, which will use the Flight Booking Agent when needed:


In [ ]:
# Test 1: General travel question (should not trigger flight booking)
print("🧪 Test 1: General travel question")
response = await chat_with_travel_agent(
    "What are some good travel destinations for summer?",
    "demo_context"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

In [ ]:
# Test 2: Flight booking request (should trigger A2A communication)
print("🧪 Test 2: Flight booking request (A2A communication)")
response = await chat_with_travel_agent(
    "I need to book a flight from San Francisco to Tokyo on December 25th",
    "demo_context"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

In [ ]:

    # Test 3: Follow-up with more flight details (should continue A2A communication)
print("🧪 Test 3: Follow-up with more flight details")
response = await chat_with_travel_agent(
    "Make it for 2 passengers, and I prefer business class",
    "demo_context"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

### Understanding the Test Results

Let's analyze what happened in our A2A communication tests:

#### **Test 1: General Travel Question**
- **No A2A Call**: The travel agent handled this directly without calling the flight booking agent
- **Direct Response**: It provided travel destination recommendations from its own knowledge
- **Smart Routing**: It correctly identified this as a general travel question, not a flight booking request

#### **Test 2: Flight Booking Request**
- **A2A Triggered**: The travel agent detected flight booking intent and called the flight booking tool
- **Protocol Communication**: We can see the HTTP requests to the flight booking agent in the logs
- **Agent Card Fetching**: The system fetched the agent card to understand capabilities
- **Message Routing**: The request was properly formatted and sent to the flight booking agent
- **Response Handling**: The flight booking agent's response was returned through the travel agent

#### **Test 3: Follow-up Details**
- **Continued A2A**: The travel agent again used A2A to send additional details
- **Context Preservation**: The flight booking agent maintained context from the previous interaction
- **Successful Completion**: The booking was completed with all necessary information

This demonstrates the power of A2A communication - agents can specialize while working together seamlessly!


In [ ]:
print("🧪 Test 4: Mixed request (travel advice + flight booking)")
response = await chat_with_travel_agent(
    "What's the best time to visit Paris? Also, can you book me a flight there for next month?",
    "demo_context2"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

In [ ]:
print("🧪 Test 5: Follow up to finalize the booking)")
response = await chat_with_travel_agent(
    "It's going to be just for myself, departing from Cairo on the 15th of November for 5 days and lets do Economy class",
    "demo_context2"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

### Advanced A2A Features Demonstrated

Our tests showcase several advanced A2A capabilities:

#### **Test 4: Mixed Request Handling**
- **Hybrid Processing**: The travel agent handled both travel advice and flight booking in one request
- **Intelligent Routing**: It provided travel advice directly and initiated A2A for flight booking
- **Context Switching**: It managed both general knowledge and specialized tool usage

#### **Test 5: Multi-Turn A2A Conversations**
- **Stateful A2A**: The flight booking agent maintained context across multiple A2A calls
- **Progressive Information Gathering**: Each A2A call built upon previous interactions
- **Error Handling**: The system gracefully handled incomplete or conflicting information

#### **Key Observations:**
1. **Automatic Tool Selection**: The travel agent automatically decides when to use A2A
2. **Context Preservation**: Both agents maintain their respective conversation contexts
3. **Protocol Transparency**: The A2A communication is seamless to the end user
4. **Error Resilience**: The system handles network issues and agent unavailability gracefully


### Interactive Chat

You can now interact with the Travel Planning Agent directly. Try asking about travel destinations, flight bookings, or any travel-related questions:


In [ ]:
# Interactive Chat Function
async def interactive_chat():
    """Interactive chat with the Travel Planning Agent."""
    print("💬 Starting interactive chat with Travel Planning Agent")
    print("Type 'exit' to end the conversation")
    print("-" * 50)
    
    context_id = "interactive_session"
    
    while True:
        try:
            # In a real Jupyter environment, you might want to use input() or ipywidgets
            user_input = input("You: ")
            
            if user_input.lower() in ['exit', 'quit', 'bye']:
                print("👋 Thank you for chatting! Goodbye!")
                break
                
            if user_input.strip():
                response = await chat_with_travel_agent(user_input, context_id)
                print(f"Travel Agent: {response}")
                print("-" * 50)
                
        except KeyboardInterrupt:
            print("\n👋 Chat ended by user. Goodbye!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")
            print("-" * 50)

# Example usage (uncomment to run interactively)
# await interactive_chat()

# For demonstration, let's show some example interactions
print("💡 Example interactions you can try:")
print("- 'What are the best beaches in Thailand?'")
print("- 'Book a flight from London to New York tomorrow'")
print("- 'What's the weather like in Tokyo in spring?'")
print("- 'I need a round-trip flight to Paris for 2 people'")
print("- 'Tell me about travel insurance options'")
print("")
print("💬 To start interactive chat, uncomment and run: await interactive_chat()")


### A2A Communication Flow Diagram

Here's what happens when the travel agent needs to book a flight:

```
┌─────────────────┐    1. User Request   ┌─────────────────┐
│                 │ ───────────────────► │                 │
│      User       │                      │ Travel Planning │
│                 │ ◄─────────────────── │     Agent       │
└─────────────────┘    6. Final Response └─────────────────┘
                                                   │
                                                   │ 2. Detects Flight
                                                   │    Booking Intent
                                                   ▼
                                         ┌─────────────────┐
                                         │                 │
                                         │  FlightBooking  │
                                         │     Tool        │
                                         │                 │
                                         └─────────────────┘
                                                   │
                                                   │ 3. A2A Protocol
                                                   │    Communication
                                                   ▼
                                         ┌─────────────────┐
                                         │                 │
                                         │ Flight Booking  │
                                         │     Agent       │
                                         │   (A2A Server)  │
                                         │                 │
                                         └─────────────────┘
                                                   │
                                                   │ 4. Processes Request
                                                   │ 5. Returns Response
                                                   ▼
```

This architecture allows for:
- **Separation of Concerns**: Each agent has a specific responsibility
- **Reusability**: The flight booking agent can be used by multiple travel agents
- **Scalability**: Agents can be deployed and scaled independently
- **Maintainability**: Changes to flight booking logic don't affect travel planning logic



## Summary and Key Concepts

Congratulations! You've successfully implemented Agent-to-Agent (A2A) communication using Semantic Kernel. Here's what we accomplished:

### 🎯 What We Built

1. **Flight Booking Agent**: A specialized agent that handles flight bookings
2. **A2A Server**: Exposes the flight booking agent via HTTP API
3. **Travel Planning Agent**: A general travel agent that uses the flight booking agent as a tool
4. **A2A Communication**: Seamless communication between agents

### 🔑 Key Concepts Learned

#### 1. **Agent Specialization**
- Each agent has a specific purpose and set of capabilities
- Specialization leads to better performance and maintainability
- Agents can be developed and deployed independently

#### 2. **A2A Protocol**
- Standardized way for agents to communicate
- Includes agent cards, skill definitions, and message formats
- Enables discovery and invocation of agent capabilities

#### 3. **Tool Integration**
- Agents can use other agents as tools
- Semantic Kernel's function calling mechanism works with A2A
- Allows for complex workflows and orchestration

#### 4. **Context Management**
- Each agent maintains its own conversation context
- Context IDs ensure proper conversation tracking
- Enables stateful interactions across agent boundaries

### 🏗️ Architecture Benefits

1. **Modularity**: Each agent can be developed, tested, and deployed separately
2. **Scalability**: Agents can be scaled independently based on demand
3. **Reusability**: Specialized agents can be used by multiple orchestrators
4. **Maintainability**: Clear separation of concerns makes code easier to maintain

### 🚀 Next Steps

To extend this implementation, you could:

1. **Add More Specialized Agents**: Hotel booking, car rental, restaurant recommendations
2. **Add More Tools**: Integrate with real booking APIs and services

This tutorial demonstrates the power of agent-to-agent communication in creating sophisticated, distributed AI systems. The modular approach allows for building complex workflows while maintaining clean, maintainable code."


## Optional: Cleanup

If you want to stop the A2A server that's running in the background, you can run the following code:


In [ ]:
# Optional: Cleanup
# Note: The server thread is running as a daemon thread, so it will automatically
# terminate when the main Python process ends. If you want to explicitly check
# if the server is still running, you can use:

if server_thread.is_alive():
    print("✅ A2A Server is still running in the background")
    print("🔧 The server will automatically stop when the notebook kernel is restarted")
else:
    print("❌ A2A Server thread has stopped")

# You can also check the server status again
print("\n🔍 Final server status check:")
try:
    async with httpx.AsyncClient() as client:
        response = await client.get(f"http://localhost:{SERVER_PORT}/.well-known/agent.json")
        if response.status_code == 200:
            print("✅ A2A Server is still responding")
        else:
            print(f"❌ A2A Server returned status code: {response.status_code}")
except Exception as e:
    print(f"❌ Cannot connect to A2A Server: {e}")

print("\n🎉 Tutorial completed successfully!")
print("💡 Remember: The A2A server runs as a daemon thread and will stop automatically when the notebook kernel is restarted.")


## Running the Standalone Agent Files

This tutorial includes standalone implementations of both agents that can be run independently:

### Flight Booking Agent Server
The flight booking agent is available as a standalone server in the `flight-booking-agent/` directory:

```bash
cd flight-booking-agent/
uv run server.py
```

This will start the A2A server on `http://localhost:9999` with the flight booking agent.

### Travel Booking Agent
The travel booking agent with a web interface is available in the `travel-booking-agent/` directory:

```bash
cd travel-booking-agent/
uv run agent.py
```

This will start a local web server where you can interact with the travel planning agent through a browser interface.